In [4]:
import pandas as pd
import os
import sys

In [7]:
notebooks_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebooks_dir, os.pardir))

FileNotFoundError: [Errno 2] No such file or directory

In [6]:
# Get the parent directory of the current working directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Add the parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
# Get the current working directory (where the notebook is located)
current_dir = os.getcwd()

# Assuming the utils folder is in the same directory as the notebook
utils_dir = os.path.join(current_dir, 'utils')

In [ ]:
# Paths
expression_data_path = f"{os.getcwd()}/data/data_expression"
upstream_data_path = (
    f"{os.getcwd()}/data/data_sequences_upstream/upstream_sequences.xlsx"
)
merged_data_path = f"{os.getcwd()}/data/merged_data.csv"

# Get expression data
expression_df = get_expression_data(expression_data_path)

# Get upstream data
upstream_df = get_upstream_data(upstream_data_path)

# Get merged data
merged_df = get_merged_data(expression_df, upstream_df)

# Save the merged data to a CSV file
merged_df.to_csv(merged_data_path, index=False)
print(f"Merged data saved to {merged_data_path}")

In [ ]:
# Ensure consistency with the expression data
assert (
    len(set(upstream_df["csv"]).difference(set(expression_df["csv"]))) == 0
), "Inconsistent CSV filenames between upstream and expression data."
assert (
    len(
        set(map(tuple, upstream_df[["csv", "region"]].values)).difference(
            set(map(tuple, expression_df[["csv", "region"]].values))
        )
    )
    == 0
), "Inconsistent 'csv' and 'region' combinations between upstream and expression data."


In [ ]:
def find_conflicting_chromosomes(df: DataFrame) -> DataFrame:
    """
    Find regions corresponding to the same region and species but with different chromosome values.

    Args:
    df (DataFrame): The input DataFrame with columns 'species', 'chromosome', and 'region'.

    Returns:
    DataFrame: A DataFrame containing rows where there are different 'chromosome' values
               for the same 'species' and 'region'.
    """
    # Group by 'species' and 'region', then filter groups with more than one unique 'chromosome'
    result = df.groupby(['species', 'region']).filter(lambda x: x['chromosome'].nunique() > 1)
    return result